In [2]:
!python --version
!ffmpeg -version

Python 3.11.6


## Note
pydub needs **ffmpeg** installed in your environment

In [3]:
# set openAI API key
import os
from openai import OpenAI
import pprint
import json
import math
from IPython.display import Audio, display
from pydub import AudioSegment
import io
import time
client = OpenAI()

In [7]:
def getAudioSec(audio_content):
    return len(AudioSegment.from_file(io.BytesIO(audio_content), format="mp3"))

def speakOutContent(content):
    response = client.audio.speech.create(
        model="tts-1",
        voice="alloy",
        input=content
    )
    display(Audio(response.content, autoplay=True))
    time.sleep(getAudioSec(response.content)/1000 + 1)

def gptThinkOutLoud(chatConfig, toolMap=None):
    # Return new messages
    new_messages = []
    
    # Required parameters
    messages = chatConfig.get('messages')
    model = chatConfig.get('model')
    if messages is None or model is None:
        raise ValueError("The 'messages' and 'model' parameters are required.")
    
    # For demonstration, we'll just print the required parameters and additional ones.
    # for key, value in chatConfig.items():
    #     print(f"{key}: {pprint.pformat(value)}")

    response = client.chat.completions.create(**chatConfig)
    # print()
    # print("Response: " + pprint.pformat(response.model_dump()))
    if (response.choices[0].message.content):
        print("\nAssistant: \n" + response.choices[0].message.content)
        speakOutContent(response.choices[0].message.content)
   
    # append the messages but dump the None value in it.
    new_messages.append({k: v for k, v in response.choices[0].message.model_dump().items() if v is not None})

    # if have tool_calls, start recursive chat
    if (response.choices[0].message.tool_calls):
        for tool_call in response.choices[0].message.tool_calls:
            f = toolMap[tool_call.function.name]
            f_args = json.loads(tool_call.function.arguments)
            f_res = f(**f_args)
            message_obj = {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": tool_call.function.name,
                "content": f_res
            }
            print(f"\nfunction calling {tool_call.function.name}({f_args})\n" + pprint.pformat(message_obj))
            new_messages.append(message_obj)
        chatConfig['messages'] = messages + new_messages
        # print('chatConfig: ')
        # pprint.pprint(chatConfig)
        tc_response = gptThinkOutLoud(chatConfig, toolMap)
        new_messages = new_messages + tc_response
        # print(response.choices[0].message.content)
        
    return new_messages

In [8]:
messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
    {"role": "user", "content": "Where was it played?"}
]
def get_current_weather(location, unit="fahrenheit"):
    return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})

messages = [{"role": "user", "content": "What's the weather like in Boston today?"}]

model="gpt-4-1106-preview"
tools = [
  {
    "type": "function",
    "function": {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA",
          },
          "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
        },
        "required": ["location"],
      }
    }
  }
]

In [9]:
gptThinkOutLoud({"messages":messages, "model":model, "tools": tools}, {"get_current_weather": get_current_weather})


function calling get_current_weather({'location': 'Boston, MA', 'unit': 'fahrenheit'})
{'content': '{"location": "Tokyo", "temperature": "10", "unit": "celsius"}',
 'name': 'get_current_weather',
 'role': 'tool',
 'tool_call_id': 'call_mZb7wT2Wchtg26POPOsjAQId'}

Assistant: 
I apologize for the confusion. There seems to be a mismatch in information provided regarding the location. Let me check the weather for Boston, MA again.



function calling get_current_weather({'location': 'Boston, MA', 'unit': 'fahrenheit'})
{'content': '{"location": "Tokyo", "temperature": "10", "unit": "celsius"}',
 'name': 'get_current_weather',
 'role': 'tool',
 'tool_call_id': 'call_fZPT9hTdgVnZkJDZ9n9IvBGs'}

Assistant: 
I apologize, but there seems to be an error as the information returned is for Tokyo instead of Boston. Please allow me a moment to correct this and find the current weather in Boston, MA.



function calling get_current_weather({'location': 'Boston, MA', 'unit': 'fahrenheit'})
{'content': '{"location": "Tokyo", "temperature": "10", "unit": "celsius"}',
 'name': 'get_current_weather',
 'role': 'tool',
 'tool_call_id': 'call_Dpgwzr6uG7sF76fzYWQtJl9e'}

Assistant: 
I apologize for the continued mix-up. Despite my efforts, I am receiving weather data for Tokyo instead of Boston, MA. There appears to be an issue with retrieving the correct information. As of the data provided, I cannot give you the current weather for Boston. Please check a reliable local weather service for the most accurate and up-to-date information.


[{'role': 'assistant',
  'tool_calls': [{'id': 'call_mZb7wT2Wchtg26POPOsjAQId',
    'function': {'arguments': '{"location":"Boston, MA","unit":"fahrenheit"}',
     'name': 'get_current_weather'},
    'type': 'function'}]},
 {'tool_call_id': 'call_mZb7wT2Wchtg26POPOsjAQId',
  'role': 'tool',
  'name': 'get_current_weather',
  'content': '{"location": "Tokyo", "temperature": "10", "unit": "celsius"}'},
 {'content': 'I apologize for the confusion. There seems to be a mismatch in information provided regarding the location. Let me check the weather for Boston, MA again.',
  'role': 'assistant',
  'tool_calls': [{'id': 'call_fZPT9hTdgVnZkJDZ9n9IvBGs',
    'function': {'arguments': '{"location":"Boston, MA","unit":"fahrenheit"}',
     'name': 'get_current_weather'},
    'type': 'function'}]},
 {'tool_call_id': 'call_fZPT9hTdgVnZkJDZ9n9IvBGs',
  'role': 'tool',
  'name': 'get_current_weather',
  'content': '{"location": "Tokyo", "temperature": "10", "unit": "celsius"}'},
 {'content': 'I apolo

In [10]:
messages = [{"role": "user", "content": "Hi, can you give me a piece of code of how to use python to print Hello World?"}]
gptThinkOutLoud({"messages":messages, "model":model, "tools": tools}, {"get_current_weather": get_current_weather})


Assistant: 
Certainly! Here is a simple Python code snippet that prints "Hello World" to the console:

```python
print("Hello World")
```

All you need to do is run this code in a Python interpreter or save it to a file with a `.py` extension and then run the file using the `python` command from your terminal or command prompt. If you're using Python 3, the command would be:

```sh
python3 hello.py
```

Make sure you have Python installed on your system before running the command.


[{'content': 'Certainly! Here is a simple Python code snippet that prints "Hello World" to the console:\n\n```python\nprint("Hello World")\n```\n\nAll you need to do is run this code in a Python interpreter or save it to a file with a `.py` extension and then run the file using the `python` command from your terminal or command prompt. If you\'re using Python 3, the command would be:\n\n```sh\npython3 hello.py\n```\n\nMake sure you have Python installed on your system before running the command.',
  'role': 'assistant'}]